## Домашнее задание 1
Проанализируйте данные и найдите самый холодный и самый ветренный дни в Сегеде за всю имеющуюся историю.

Узнайте сколько раз шел дождь летом 2014 года.

Сгруппируйтей данные по годам, месяцам и неделям (по отдельности) и найдите:

1) самый ветрянный месяц
2) самый влажный год
3) неделю с самой переменчивой погодой

Для работы с датами вам пригодится [pd.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

Ещё полезно будет пройтись по первому заниятию из [ml_course.ai](https://mlcourse.ai/) - https://habr.com/ru/company/ods/blog/322626/

In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
data = pd.read_csv('weatherHistory.csv')

In [4]:
data.columns

Index(['Formatted Date', 'Summary', 'Precip Type', 'Temperature (C)',
       'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Loud Cover',
       'Pressure (millibars)', 'Daily Summary'],
      dtype='object')

In [5]:
def exclude_ps(text):
    if '(' in text:
        return False
    else:
        return True

def fix_column(column):
    column = column.lower()
    column = [word for word in column.split() if exclude_ps(word)]
    column = '_'.join(column)
    
    return column

replacement = {}
for column in data.columns:
    replacement[column] = fix_column(column)
data.rename(replacement, axis=1, inplace=True)

In [6]:
data.rename({'formatted_date': 'date'}, axis=1, inplace=True)

In [7]:
data.columns

Index(['date', 'summary', 'precip_type', 'temperature', 'apparent_temperature',
       'humidity', 'wind_speed', 'wind_bearing', 'visibility', 'loud_cover',
       'pressure', 'daily_summary'],
      dtype='object')

In [8]:
data.date = data.date.str.slice(start=0, stop=-6)

In [9]:
data.date = pd.to_datetime(data.date)

In [10]:
data.sort_values(by='date', inplace=True)

In [11]:
data.head(5)

,date,summary,precip_type,temperature,apparent_temperature,humidity,wind_speed,wind_bearing,visibility,loud_cover,pressure,daily_summary
2880,2006-01-01 00:00:00,Partly Cloudy,rain,0.577778,-4.050000,0.89,17.1143,140.0,9.9820,0.0,1016.66,Mostly cloudy throughout the day.
2881,2006-01-01 01:00:00,Mostly Cloudy,rain,1.161111,-3.238889,0.85,16.6152,139.0,9.9015,0.0,1016.15,Mostly cloudy throughout the day.
2882,2006-01-01 02:00:00,Mostly Cloudy,rain,1.666667,-3.155556,0.82,20.2538,140.0,9.9015,0.0,1015.87,Mostly cloudy throughout the day.
2883,2006-01-01 03:00:00,Overcast,rain,1.711111,-2.194444,0.82,14.4900,140.0,9.9015,0.0,1015.56,Mostly cloudy throughout the day.
2884,2006-01-01 04:00:00,Mostly Cloudy,rain,1.183333,-2.744444,0.86,13.9426,134.0,9.9015,0.0,1014.98,Mostly cloudy throughout the day.


### Проанализируйте данные и найдите самый холодный и самый ветренный дни в Сегеде за всю имеющуюся историю.

Если считать, что самый ветренный/холодный тот день, в который значение соответствующего признака приняло своё максимальное/минимальное значение.

*Самый холодный*

In [12]:
data.loc[data.temperature == data.temperature.min(), ['date', 'temperature']]

,date,temperature
54847,2012-02-10 07:00:00,-21.822222


*Самый ветренный*

In [13]:
data.loc[data.wind_speed == data.wind_speed.max(), ['date', 'wind_speed']]

,date,wind_speed
12181,2007-01-29 13:00:00,63.8526


Если считать, что речь идет о *среднесуточных* температуре и скорости ветра.

In [14]:
by_day = data.groupby(by=[data.date.dt.month, data.date.dt.day, data.date.dt.year])['date', 'temperature', 'wind_speed'].mean()

*Самый холодный*

In [15]:
by_day.loc[by_day.temperature == by_day.temperature.min()]

,,,temperature,wind_speed
date,date,date,,
2,10,2012,-15.773611,4.4275


*Самый ветренный*

In [16]:
by_day.loc[by_day.wind_speed == by_day.wind_speed.max()]

,,,temperature,wind_speed
date,date,date,,
1,27,2008,6.269907,36.002954


### Узнайте сколько раз шел дождь летом 2014 года.

In [17]:
summer = data.loc[(data.date.dt.month > 5) & (data.date.dt.month < 9) & (data.date.dt.year == 2014)]

Считаем сколько *дней* шел дождь хотя бы раз.

In [18]:
summer.loc[summer.precip_type == 'rain'].date.dt.dayofyear.nunique()

92

#### 1) самый ветрянный месяц

In [19]:
by_month = data.groupby(by=data.date.dt.month)['date', 'wind_speed'].mean()

In [20]:
by_month.loc[by_month.wind_speed == by_month.wind_speed.max()]

,wind_speed
date,
3,13.405461


#### 2) самый влажный год 

In [21]:
by_year = data.groupby(by=data.date.dt.year)['date', 'humidity'].mean()

In [22]:
by_year.loc[by_year.humidity == by_year.humidity.max()]

,humidity
date,
2010,0.796858


#### 3) неделя с самой переменчивой погодой

Неделей с переменчивой погодой будем считать неделю, в которой больше всего менялись температура, влажность, скорость ветра. Т.е. данные признаки в наибольшей степени отклонялись от своего среднего значения.

In [23]:
by_week = data.groupby(by=data.date.dt.week)['date', 'temperature', 'humidity', 'wind_speed'].std()

Найдем такую неделю, отсортировав значения по убыванию (т.о. первая строка – искомая):

In [24]:
by_week.sort_values(by=['humidity', 'temperature', 'wind_speed'], ascending=False).head(1)

,temperature,humidity,wind_speed
date,,,
34,6.127291,0.222904,6.024446


### Домашнее задание 2

Попробуйте придумать признаки (как минимум 5), которые можно извлечь из текста, помимо тех 4, которые мы уже использовали. Обучите модель на них и проверьте, улучашается ли качество. Опишите признаки, которое придумали и почему считаете, что они должны показывать вероятность покупки.

Побробуйте подобрать параметры векторизатора (TfidfVectorizer или CountVectorizer, или обоих) и параметры алгоритма так, чтобы качество, которое у нас получилось (0.248) улучшилось. Опишите то, что попробовали.

In [17]:
data = pd.read_csv('data.csv')

#### Обучение на признаках текстов

Исходные признаки:

In [18]:
# длина текста
# data['text_len'] = data['description'].apply(len)
# количество токенов в тексте
# data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
# количество заглавных букв в тексте
# data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
# количество знаков пунктуации или других знаков (не цифр и не букв)
# перенос строки сюда по всей видимости тоже попадает
# data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

In [19]:
import nltk
import string

In [20]:
def upper_case_per(text):
    text = nltk.tokenize.word_tokenize(text, language='russian')
    text = [word for word in text if word.isalpha()]
    upper_case = [word for word in text if word.isupper()]
    if not text:
        return 0
    return len(upper_case)/len(text)

In [21]:
def avg_word_len(text):
    text = nltk.tokenize.word_tokenize(text, language='russian')
    text = [len(word) for word in text if word.isalpha()]
    if not text:
        return 0
    return sum(text)/len(text)

In [22]:
def zero_or_one(value):
    if value < 1:
        return 0
    else:
        return 1

In [23]:
def latin_letters(text):
    return sum([1 for ch in text if ch in string.ascii_lowercase])

In [24]:
def sent_num(text):
    return len(nltk.tokenize.sent_tokenize(text, language='russian'))
    

In [25]:
def avg_sent_len(text):
    text = [len(sent) for sent in nltk.tokenize.sent_tokenize(text, language='russian')]
    if not text:
        return 0
    return sum(text)/len(text)

Возможно, объявления, в которых много слов написано заглавными буквами могут быть менее привлекательно для потенциального покупателя.

In [26]:
data['upper_case_per'] = data.description.apply(upper_case_per)

Кажется, что объявление, полностью состоящее из заглавных букв должно отталкивать.

In [27]:
data['all_caps'] = data.upper_case_per.apply(zero_or_one)

Предположительно, описания, содержащие больше длинных слов должны быть более информативны, что может влиять на вероятность покупки.

In [28]:
data['avg_word_len'] = data.description.apply(avg_word_len)

Если объявление содержит много латинских букв, то вероятно, что в нём описываются технические характерстики продаваемой вещи. Это может влиять на вероятность покупки.

In [29]:
data['latin_letters'] = data.description.apply(latin_letters)

Возможно, описание, содержащее большее количество предложений, может показаться более структурированным и, возможно, более привлекательным.

In [30]:
data['sent_num'] = data.description.apply(sent_num)

Средняя длина предложений опять же может указывать на информативность объявления.

In [31]:
data['avg_sent_len'] = data.description.apply(avg_sent_len)

Как видим, только средняя длинна слов более-менее существенно коррелирует с целевой переменной.

In [32]:
data.corr()

,price,item_seq_number,image_top_1,deal_probability,upper_case_per,all_caps,avg_word_len,latin_letters,sent_num,avg_sent_len
price,1.000000,0.077973,0.046540,-0.015032,-0.006553,-0.002286,0.029157,-0.003490,0.070424,0.023365
item_seq_number,0.077973,1.000000,0.093411,-0.038196,0.027008,-0.005166,0.039038,0.016230,0.099171,0.062149
image_top_1,0.046540,0.093411,1.000000,0.185397,0.081460,0.011117,0.085557,0.156175,0.087741,0.169118
deal_probability,-0.015032,-0.038196,0.185397,1.000000,0.000957,0.004474,0.041378,-0.003680,0.004417,0.007912
upper_case_per,-0.006553,0.027008,0.081460,0.000957,1.000000,0.710454,-0.066318,0.047872,0.018302,0.039722
all_caps,-0.002286,-0.005166,0.011117,0.004474,0.710454,1.000000,-0.022160,-0.014972,-0.021060,-0.013542
avg_word_len,0.029157,0.039038,0.085557,0.041378,-0.066318,-0.022160,1.000000,0.018562,0.061885,0.087737
latin_letters,-0.003490,0.016230,0.156175,-0.003680,0.047872,-0.014972,0.018562,1.000000,0.117090,0.348301
sent_num,0.070424,0.099171,0.087741,0.004417,0.018302,-0.021060,0.061885,0.117090,1.000000,-0.064198
avg_sent_len,0.023365,0.062149,0.169118,0.007912,0.039722,-0.013542,0.087737,0.348301,-0.064198,1.000000


In [33]:
X = data[['upper_case_per', 'all_caps', 'avg_word_len', 'latin_letters', 'sent_num', 'avg_sent_len']].values
y = data.deal_probability.values

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [36]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

In [37]:
clf = LinearRegression().fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [38]:
y_pred

array([0.1549894 , 0.13676318, 0.14609567, ..., 0.13760842, 0.14376354,
       0.14805246])

In [39]:
np.sqrt(mean_squared_error(y_test, y_pred))
# Значение на исходных признаках 0.2633314826860365

0.2633898614650038

Полученный результат несколько ухудшился.

#### Обучение на векторном представлении текстов

In [46]:
data = pd.read_csv('data.csv')

In [89]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import r2_score, mean_squared_error

In [90]:
from nltk.corpus import stopwords

In [94]:
def eval_result(y_test, y_pred):
    print(f'{np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'{r2_score(y_test, y_pred)}')

In [150]:
def eval_model(vec_repr):
    X = vec_repr.fit_transform(data.description)
    y = data.deal_probability.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    regressor = Ridge(alpha=1).fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    eval_result(y_test, y_pred)

Использование стоп-слов не дало ощутимого прироста.

In [151]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.2514558627097011
0.0905340754682128


Повышение количества фич также не помогает.

In [153]:
cv = CountVectorizer(max_features=5000, min_df=5, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.2527414999858558
0.08121051211722075


Изменение минимальной частоты документов также не повышает качество предсказания.

In [154]:
cv = CountVectorizer(max_features=5000, min_df=3, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.2527401898318976
0.08122003768040997


In [155]:
cv = CountVectorizer(max_features=5000, min_df=7, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.2527388159880188
0.0812300262524479


Максимальная частота вообще не влияет.

In [156]:
cv = CountVectorizer(max_features=5000, min_df=5, max_df=0.2, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.2527414999858558
0.08121051211722075


In [157]:
cv = CountVectorizer(max_features=5000, min_df=5, max_df=0.7, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.2527414999858558
0.08121051211722075


Отказ от использования триграмов несколько повысил качество.

In [158]:
cv = CountVectorizer(max_features=5000, min_df=5, max_df=0.7, ngram_range=(1,2),
                     stop_words=stopwords.words('russian'))
eval_model(cv)

0.25266534445788547
0.08176412406307465


Использование стоп-слов в модели Tf-Idf кажется более полезным.

In [163]:
tf = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.4, stop_words=stopwords.words('russian'))
eval_model(tf)

0.2483492774474999
0.11286706674186087


Повышение количества фич также улучшает качество.

In [164]:
tf = TfidfVectorizer(max_features=7000, min_df=5, max_df=0.4, stop_words=stopwords.words('russian'))
eval_model(tf)

0.24772994342604907
0.11728621807444306


Увеличение минимально частоты документов оказалось полезным.

In [165]:
tf = TfidfVectorizer(max_features=7000, min_df=3, max_df=0.4, stop_words=stopwords.words('russian'))
eval_model(tf)

0.24774394329127347
0.11718644633917663


In [166]:
tf = TfidfVectorizer(max_features=7000, min_df=7, max_df=0.4, stop_words=stopwords.words('russian'))
eval_model(tf)

0.2477109359427215
0.11742166818269262


Максимальная частота похоже ничего не меняет.

In [167]:
tf = TfidfVectorizer(max_features=7000, min_df=7, max_df=0.2, stop_words=stopwords.words('russian'))
eval_model(tf)

0.2477109359427215
0.11742166818269262


In [168]:
tf = TfidfVectorizer(max_features=7000, min_df=7, max_df=0.7, stop_words=stopwords.words('russian'))
eval_model(tf)

0.2477109359427215
0.11742166818269262


Использование большего количества моделей не улучшает качество.

In [169]:
tf = TfidfVectorizer(max_features=7000, min_df=7, max_df=0.2, stop_words=stopwords.words('russian'), ngram_range=(1, 2))
eval_model(tf)

0.24786959550466828
0.11629071822586978


In [170]:
tf = TfidfVectorizer(max_features=7000, min_df=7, max_df=0.2, stop_words=stopwords.words('russian'), ngram_range=(1, 3))
eval_model(tf)

0.24784038380504242
0.11649899813935427


Использование логарифма от частоты терминов даёт некоторый прирост.

In [172]:
tf = TfidfVectorizer(max_features=7000, min_df=7, max_df=0.2, stop_words=stopwords.words('russian'), sublinear_tf=True)
eval_model(tf)

0.24761835964396067
0.1180812318565474
